In [1]:
import pandas as pd

DF = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices.xlsx')
# HIST_PRICE_DF_RAW = pd.read_csv (r'/Users/joezhou/Downloads/ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
HIST_PRICE_DF = DF[DF['TickName'].isin(['CBA.AX','WES.AX'])]

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [2]:
# Freature Engineering at daily level
HIST_PRICE_DF = DF[DF['TickName'].isin(['CBA.AX','WES.AX'])].reset_index(drop=True)

#Movement relative to Open price
HIST_PRICE_DF['HIGHtakeLOW'] = (HIST_PRICE_DF['High']-HIST_PRICE_DF['Low']).abs()
HIST_PRICE_DF['CLOSEtakeOPEN'] = (HIST_PRICE_DF['Close']-HIST_PRICE_DF['Open']).abs()
HIST_PRICE_DF['HIGHdivideLOW'] = HIST_PRICE_DF['High']/HIST_PRICE_DF['Low']-1
HIST_PRICE_DF['CLOSEdivideOPEN'] = HIST_PRICE_DF['Close']/HIST_PRICE_DF['Open']-1

HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['V_CloseHigher'] ] = HIST_PRICE_DF['CLOSEdivideOPEN']
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] < HIST_PRICE_DF['Open'] , ['V_CloseLower'] ] = HIST_PRICE_DF['CLOSEdivideOPEN']

HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['C_CloseHigher'] ] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] < HIST_PRICE_DF['Open'] , ['C_CloseLower'] ] = 1

# Create flags for aggregation
HIST_PRICE_DF['Freq'] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['C_CLOSEaboveOPEN'] ] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Open'] > HIST_PRICE_DF['Low'] , ['C_LOWaboveOpen'] ] = 1


# Running totals
# t = HIST_PRICE_DF.groupby(['TickName', 'Date','C_CloseHigher']).sum().groupby(level=0).cumsum().reset_index()


In [42]:
# Col = variable
# count = new column

#   df['count'] = df.groupby((df['col'] != df['col'].shift(1)).cumsum()).cumcount()+1


HIST_PRICE_DF['TickName_Plus'] = HIST_PRICE_DF['TickName'] + HIST_PRICE_DF['C_CloseHigher'].astype(str)


# HIST_PRICE_DF['count'] = HIST_PRICE_DF.groupby('TickName_Plus')['V_CloseHigher'].cumsum()
HIST_PRICE_DF['vol'] = HIST_PRICE_DF.groupby(HIST_PRICE_DF['TickName_Plus'] != HIST_PRICE_DF['TickName_Plus'].shift(1))['V_CloseHigher'].cumsum().cumsum()+0


# HIST_PRICE_DF['count'] = HIST_PRICE_DF.groupby((HIST_PRICE_DF['TickName_Plus'] != HIST_PRICE_DF['TickName_Plus'].shift(1)).cumsum())['V_CloseHigher'].cumcount()+1
HIST_PRICE_DF['count'] = HIST_PRICE_DF.groupby((HIST_PRICE_DF['TickName_Plus'] != HIST_PRICE_DF['TickName_Plus'].shift(1)).cumsum()).cumcount()+1

# df['CUMSUM_C'] = df['SUM_C'].cumsum()

t = HIST_PRICE_DF[['TickName','Date','C_CloseHigher','TickName_Plus','vol','V_CloseHigher','count']]


In [124]:


HIST_PRICE_DF_med = HIST_PRICE_DF.groupby(['TickName'])['HIGHtakeLOW','CLOSEtakeOPEN','HIGHdivideLOW','CLOSEdivideOPEN','V_CloseHigher','V_CloseLower'].median().add_suffix('_med')
HIST_PRICE_DF_Sum = HIST_PRICE_DF.groupby(['TickName'])['Freq','C_CloseHigher','C_CloseLower','C_LOWaboveOpen'].sum().add_suffix('_sum')


HIST_PRICE_DF_ADDED = pd.merge(HIST_PRICE_DF_Sum,HIST_PRICE_DF_med,on='TickName')


# Calculate probability of increase
HIST_PRICE_DF_ADDED['P_CloseHigher'] = HIST_PRICE_DF_ADDED['C_CloseHigher_sum'] / HIST_PRICE_DF_ADDED['Freq_sum'] 
HIST_PRICE_DF_ADDED['Mvmt_Likihood'] = (HIST_PRICE_DF_ADDED['P_CloseHigher']*HIST_PRICE_DF_ADDED['V_CloseHigher_med'])+((1-HIST_PRICE_DF_ADDED['P_CloseHigher'])*HIST_PRICE_DF_ADDED['V_CloseLower_med'])



<ipython-input-124-6df2220db16e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_med = HIST_PRICE_DF.groupby(['TickName'])['HIGHtakeLOW','CLOSEtakeOPEN','HIGHdivideLOW','CLOSEdivideOPEN','V_CloseHigher','V_CloseLower'].median().add_suffix('_med')
<ipython-input-124-6df2220db16e>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_Sum = HIST_PRICE_DF.groupby(['TickName'])['Freq','C_CloseHigher','C_CloseLower','C_LOWaboveOpen'].sum().add_suffix('_sum')


In [ ]:
#Feature Engineering at Ticker level

# BASELINE = BASELINE.columns = ['TickerName']
PRICE_MEDIAN = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].median()
PRICE_STD_DEV = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].std()


TICKER_LIST_ADDED = pd.merge(TICKER_LIST,PRICE_MEDIAN,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_MEDIAN'})


TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,PRICE_STD_DEV,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_STD'})



In [ ]:


TICKER_LIST_ADDED.to_excel('/Users/joezhou/Downloads/ALL_Prices_Features.xlsx')  
